# Librairies

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from matplotlib import pyplot as plt

# Objectifs

A partir des données de transactions financières, créer un modèle prédictif du prix d’un appartement en fonction au moins des paramètres suivants :
- Commune
- Superficie
- Nombre de pièces
- Année de transaction
- Voir plus...

# Nettoyage de donnée

In [2]:
file = os.listdir('./Data')[0]
file_path = os.path.join('./Data', file)
df1 = pd.read_csv(file_path, sep='|')
print(list(df1.columns))


C:\Users\YLK\AppData\Local\Temp/ipykernel_1344/3446110912.py:3: DtypeWarning: Columns (18,23,24,26,28,30,31,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(file_path, sep='|')


['Code service CH', 'Reference document', '1 Articles CGI', '2 Articles CGI', '3 Articles CGI', '4 Articles CGI', '5 Articles CGI', 'No disposition', 'Date mutation', 'Nature mutation', 'Valeur fonciere', 'No voie', 'B/T/Q', 'Type de voie', 'Code voie', 'Voie', 'Code postal', 'Commune', 'Code departement', 'Code commune', 'Prefixe de section', 'Section', 'No plan', 'No Volume', '1er lot', 'Surface Carrez du 1er lot', '2eme lot', 'Surface Carrez du 2eme lot', '3eme lot', 'Surface Carrez du 3eme lot', '4eme lot', 'Surface Carrez du 4eme lot', '5eme lot', 'Surface Carrez du 5eme lot', 'Nombre de lots', 'Code type local', 'Type local', 'Identifiant local', 'Surface reelle bati', 'Nombre pieces principales', 'Nature culture', 'Nature culture speciale', 'Surface terrain']


In [3]:
files = [os.listdir('./Data')][0]
columns_type_fills = np.array([
    ['Date mutation', np.int16, -1],
    ['Commune', 'string', ''],
    ['Nature mutation', 'string', ''],
    ['Nature culture', 'string', ''],
    ['Nombre de lots', np.int8, -1],
    ['Nombre pieces principales', np.int8, -1],
    ['Surface reelle bati', np.float32, -1],
    ['Surface terrain', np.float32, -1],
    ['Valeur fonciere', np.float32,  -1]
    ])
columns,types,fills = [list(columns_type_fills[:,i]) for i in range(columns_type_fills.shape[1])]
columns_types = dict(zip(columns,types))
df = []


In [4]:
for file in files:
    file_path = os.path.join('./Data', file)
    temp_df = pd.read_csv(file_path, sep='|', usecols=columns+['Code type local'])
    temp_df = temp_df[temp_df['Code type local'].eq(2.0)]
    temp_df['Date mutation'] = temp_df['Date mutation'].str.strip().str[-4:]
    temp_df['Valeur fonciere'] = temp_df['Valeur fonciere'].str.replace(',','.')
    for column, fill in zip(columns,fills) :
        temp_df[column].fillna(fill, inplace=True)
    temp_df = temp_df[columns].astype(columns_types)
    df.append(temp_df)
    
big_data = pd.concat(df)

In [5]:
big_data.head()

,Date mutation,Commune,Nature mutation,Nature culture,Nombre de lots,Nombre pieces principales,Surface reelle bati,Surface terrain,Valeur fonciere
9,2016,BOURG-EN-BRESSE,Vente,,4,2,79.0,-1.0,111700.0
58,2016,BOURG-EN-BRESSE,Vente,,1,5,133.0,-1.0,311700.0
67,2016,SAINT-DENIS-LES-BOURG,Vente,,2,2,42.0,-1.0,65000.0
79,2016,NEUVILLE-SUR-AIN,Vente,,5,3,78.0,-1.0,158240.0
82,2016,SAINT-LAURENT-SUR-SAONE,Vente,,1,2,55.0,-1.0,77000.0


In [6]:
big_data.describe()

,Date mutation,Nombre de lots,Nombre pieces principales,Surface reelle bati,Surface terrain,Valeur fonciere
count,2.835622e+06,2.835622e+06,2.835622e+06,2.835622e+06,2.835622e+06,2.835622e+06
mean,2.018364e+03,1.170345e+00,2.590810e+00,5.695084e+01,9.781821e+02,2.184380e+06
std,1.427267e+00,1.020591e+00,1.191713e+00,2.943501e+01,1.228858e+04,1.527559e+07
min,2.016000e+03,0.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00
25%,2.017000e+03,1.000000e+00,2.000000e+00,3.700000e+01,-1.000000e+00,1.040000e+05
50%,2.018000e+03,1.000000e+00,3.000000e+00,5.500000e+01,-1.000000e+00,1.750000e+05
75%,2.019000e+03,2.000000e+00,3.000000e+00,7.100000e+01,-1.000000e+00,3.266000e+05
max,2.021000e+03,8.500000e+01,9.000000e+01,2.426000e+03,4.301668e+06,1.256966e+09


In [7]:
print(big_data.dtypes)
nb_rows, nb_cols = big_data.shape
print('Nb de lignes :',nb_rows, ', Nb de col :', nb_cols)
print('Nb de communes :', big_data[['Commune']].nunique().values[0])

Date mutation                  int16
Commune                       string
Nature mutation               string
Nature culture                string
Nombre de lots                  int8
Nombre pieces principales       int8
Surface reelle bati          float32
Surface terrain              float32
Valeur fonciere              float32
dtype: object
Nb de lignes : 2835622 , Nb de col : 9
Nb de communes : 15829


# Préparation du dataset

In [8]:
list_le = [preprocessing.LabelEncoder()]*3
data_set = big_data.copy()
for col, le in zip(['Commune','Nature mutation','Nature culture'],list_le):
    data_set[col] = le.fit_transform(data_set[col])
data_set.head()

,Date mutation,Commune,Nature mutation,Nature culture,Nombre de lots,Nombre pieces principales,Surface reelle bati,Surface terrain,Valeur fonciere
9,2016,1801,3,0,4,2,79.0,-1.0,111700.0
58,2016,1801,3,0,1,5,133.0,-1.0,311700.0
67,2016,12083,3,0,2,2,42.0,-1.0,65000.0
79,2016,9656,3,0,5,3,78.0,-1.0,158240.0
82,2016,12537,3,0,1,2,55.0,-1.0,77000.0


In [9]:
def train_valid_teset_split (dataX, dataY,train_ratio = 0.8, validation_ratio = 0.1, test_ratio = 0.1):
    x_train, x_test, y_train, y_test = train_test_split(dataX, dataY, test_size=1 - train_ratio, random_state=111)
    x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio), random_state=111) 

    return x_train, x_val, x_test, y_train, y_val, y_test

dataY, dataX = data_set.pop('Valeur fonciere'), data_set
x_train, x_val, x_test, y_train, y_val, y_test = train_valid_teset_split (dataX, dataY)

# Modèle de prédiction

In [10]:
rfr = RandomForestRegressor(n_estimators=500, criterion='mse', n_jobs=-1, random_state=111, max_samples=0.3, verbose=True) # max_depth=10, max_leaf_nodes=15829,
rfr.fit(x_train, y_train)
print(rfr.score(x_val, y_val))
print(rfr.score(x_test, y_test))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:   24.6s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  6.6min finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    3.7s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:  4.1min
[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed: 12.9min
[Parallel(n_jobs=16)]: Done 500 out of 500 | elapsed: 13.0min finished


0.7348662338803953


[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    6.8s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:   49.4s
[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed:  2.4min


0.8939879968710149


[Parallel(n_jobs=16)]: Done 500 out of 500 | elapsed:  2.5min finished


# Evaluation

In [12]:
rfr.score(x_train, y_train)

[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    9.2s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:   58.7s
[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed:  2.5min
[Parallel(n_jobs=16)]: Done 500 out of 500 | elapsed:  2.9min finished


0.8880843081357028

In [14]:
for i in range(2016,2022):
    columns_values = pd.DataFrame({
        'Date mutation': [i],
        'Commune': ['PARIS 08'],
        'Nature mutation': ['Vente'],
        'Nature culture': [''],
        'Nombre de lots': [0],
        'Nombre pieces principales': [1],
        'Surface reelle bati':[ 14],
        'Surface terrain': [28],
        'Valeur fonciere': ['202000'],
        })
    for col, le in zip(['Commune','Nature mutation','Nature culture'],list_le):
        columns_values[col] = le.fit_transform(columns_values[col])
    Y, X = columns_values.pop('Valeur fonciere'), columns_values
    print(rfr.predict(X))

[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed:    0.1s
[Parallel(n_jobs=16)]: Done 500 out of 500 | elapsed:    0.1s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.


[192560.02133333]
[180166.10619048]


[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 500 out of 500 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 500 out of 500 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 500 out of 500 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 con

[182485.98353497]
[164157.49842247]


[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:    0.0s


[172697.12608854]
[171820.04123438]


[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed:    1.2s
[Parallel(n_jobs=16)]: Done 500 out of 500 | elapsed:    1.2s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 500 out of 500 | elapsed:    0.1s finished
